In [77]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests

print('Libraries imported.')

Libraries imported.


In [3]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

# 1. Creating the DataFrame of Canadian Postalcodes

### I install the beautifulsoup library to scrape the table from the wikipedia page.

In [4]:
!pip install beautifulsoup4

     |████████████████████████████████| 122kB 6.9MB/s eta 0:00:01


### Note
>I import the beautifulSoup class from bs4 and with the request library get the underlying html of the page. I then create a beautifulsoup object and print a prettified version of it (I will only a part of this so scrolling doesn't become a problem). I scanned through this html that i called soup and found the class pointing to the table.

In [5]:
from bs4 import BeautifulSoup

url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(url)

print(soup.prettify()[:500])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"3799


### Note:
>I then use the find method to extract the portion of the html relating to the table and the findAll and text function to take out the text from the html into variables header(a list) and body(a list of lists)

In [6]:
table= soup.find('table',{'class':'wikitable sortable'})

headers = [j.text.rstrip() for j in table.findAll('tr')[0].findAll('th')]
body= [[j.text.rstrip() for j in i.findAll('td')] for i in table.findAll('tr')[1:]]

### I use the filter method to extract all rows that have Boroughs. I also check and verify that no row with a borough has a Not assigned Neighborhood (Hence there is no need to process anything as regards that).

In [7]:
new_body = filter(lambda x: x[1]!= 'Not assigned',body)
print(f'Number of cells with a borough but a Not assigned Neighborhoods:{len(list(filter(lambda x: x[1] != "Not assigned" and x[2]== "Not assigned",body)))}')

Number of cells with a borough but a Not assigned Neighborhoods:0


### I loaded my data with its columns into a dataframe object.

In [8]:
neigh_df = pd.DataFrame(new_body,columns=headers)
neigh_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Finally, I output the shape of the dataframe.

In [9]:
neigh_df.shape

(103, 3)

# 2. Adding the Geographical Data

### I used the pd.read_csv method to read the csv data from the link provided and check its shape.

In [10]:
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')
geo_df.shape

(103, 3)

### I used the Pandas merge method to merge the neighbourhood data and the geographical data into one dataframe called df.

In [11]:
df = pd.merge(neigh_df,geo_df,on = 'Postal Code')
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# 3. Exploring and clustering the neighborhoods in Toronto

### I extract the indices for the rows that have Toronto in their Boroughs.

In [79]:
toronto_df = df.iloc[[i for i,j in enumerate(df['Borough']) if 'Toronto' in j],:]

In [30]:
dict(toronto_df.iloc[0,:])

{'Postal Code': 'M5A',
 'Borough': 'Downtown Toronto',
 'Neighborhood': 'Regent Park, Harbourfront',
 'Latitude': 43.6542599,
 'Longitude': -79.3606359}

### Here, I separate the different Neighborhoods in each Borough into different entries by looping through, splitting the Neighborhood by comma and stripping off leading and lagging whitespaces then appending each to the dataframe.

In [42]:
new_df = pd.DataFrame(columns = toronto_df.columns)
for i in range(toronto_df.shape[0]):
    b = toronto_df.iloc[i,:]
    format_ = {'Postal Code': b['Postal Code'],
 'Borough': b['Borough'],
 'Neighborhood':'',
 'Latitude': b['Latitude'],
 'Longitude': b['Longitude']}
    for n in [j.strip() for j in b['Neighborhood'].split(',')]:
        format_['Neighborhood']=n
        new_df = new_df.append(format_,ignore_index=True)
new_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
1,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
2,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
3,M7A,Downtown Toronto,Ontario Provincial Government,43.662301,-79.389494
4,M5B,Downtown Toronto,Garden District,43.657162,-79.378937
5,M5B,Downtown Toronto,Ryerson,43.657162,-79.378937
6,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
7,M4E,East Toronto,The Beaches,43.676357,-79.293031
8,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
9,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


### Here I create one hot encodings (dummies) of the different Boroughs in the dataset and display the first five.

In [64]:
# one hot encoding
toronto_onehot = pd.get_dummies(new_df,columns=['Borough'], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = new_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,West Toronto,Postal Code,Neighborhood,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto
0,0,M5A,Regent Park,43.654260,-79.360636,0,1,0
1,0,M5A,Harbourfront,43.654260,-79.360636,0,1,0
2,0,M7A,Queen's Park,43.662301,-79.389494,0,1,0
3,0,M7A,Ontario Provincial Government,43.662301,-79.389494,0,1,0
4,0,M5B,Garden District,43.657162,-79.378937,0,1,0


In [65]:
toronto_onehot.shape

(78, 8)

### Here I group the dataframe by Neighbourhood, calculate the mean of the different sub dataframes and make that into a new dataframe.

In [66]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,West Toronto,Latitude,Longitude,Central Toronto,Downtown Toronto,East Toronto
0,Adelaide,0,43.650571,-79.384568,0,1,0
1,Bathurst Quay,0,43.628947,-79.394420,0,1,0
2,Berczy Park,0,43.644771,-79.373306,0,1,0
3,Brockton,1,43.636847,-79.428191,0,0,0
4,Business reply mail Processing Centre,0,43.662744,-79.321558,0,0,1
5,CN Tower,0,43.628947,-79.394420,0,1,0
6,Cabbagetown,0,43.667967,-79.367675,0,1,0
7,Central Bay Street,0,43.657952,-79.387383,0,1,0
8,Chinatown,0,43.653206,-79.400049,0,1,0
9,Christie,0,43.669542,-79.422564,0,1,0


#### Here I cluster the Neighbourhoods using the Kmeans algorithm. I selected 4 clusters because it seemed to give the most reasonable visuals when the map was displayed.

In [72]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped[['West Toronto','Central Toronto','Downtown Toronto','East Toronto']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 3, 0, 0, 0, 0, 0], dtype=int32)

In [73]:
toronto_grouped['Cluster Labels']= kmeans.labels_


#### Here I get the geographical coordinates of toronto using the geopy library.

In [62]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### I then create the Map and center it around Toronto with a starting zoom of 12.

In [78]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
map_clusters

## Finally, I set the color scheme for the clusters then add markers to the map.

In [76]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped['Latitude'], toronto_grouped['Longitude'], toronto_grouped['Neighborhood'], toronto_grouped['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters